# 01 - Exploratory Data Analysis

**Purpose**: Initial exploration and understanding of the spatial transcriptomics dataset

**Contents**:
- Data loading and first look
- Basic statistics and data structure
- Initial visualizations
- Data quality assessment
- Parameter exploration for downstream analysis

**Author**: [Your Name]  
**Date**: 2025  
**Dataset**: Visium Human Breast Cancer

In [ ]:
# Import libraries
import sys
sys.path.append('../')

import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Import our custom modules
from modules.data_loader import VisiumDataLoader, load_and_validate_visium
from modules.quality_control import SpatialQualityControl

# Configure plotting
plt.rcParams['figure.figsize'] = (10, 8)
plt.rcParams['font.size'] = 12
sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=80, facecolor='white')

print("📚 Libraries imported successfully!")

## 📥 Data Loading

Let's start by loading and examining our spatial transcriptomics dataset.

In [ ]:
# Define data path
DATA_PATH = "../data/example_data/"

# Load data using our custom loader
print("🔄 Loading Visium data...")
adata, validation_results = load_and_validate_visium(
    data_path=DATA_PATH,
    fix_uniformity=True  # Auto-fix if artificial uniformity detected
)

print(f"✅ Data loaded: {adata.shape[0]} spots × {adata.shape[1]} genes")

## 🔍 First Look at the Data

Let's examine the basic structure and properties of our dataset.

In [ ]:
# Basic data information
print("📊 DATASET OVERVIEW:")
print(f"   Spots (observations): {adata.n_obs:,}")
print(f"   Genes (variables): {adata.n_vars:,}")
print(f"   ") 
print(f"📋 OBSERVATION METADATA:")
print(f"   Columns: {list(adata.obs.columns)}")
print(f"")
print(f"🧬 GENE METADATA:")
print(f"   Columns: {list(adata.var.columns)}")
print(f"")
print(f"🗂️ ADDITIONAL DATA:")
print(f"   .obsm keys: {list(adata.obsm.keys())}")
print(f"   .uns keys: {list(adata.uns.keys())}")

In [ ]:
# Display the validation results
loader = VisiumDataLoader(DATA_PATH)
loader.validation_results = validation_results
print(loader.get_validation_summary())

## 🧬 Gene Expression Overview

Let's explore the basic properties of gene expression in our dataset.

In [ ]:
# Calculate basic expression statistics
X = adata.X
if hasattr(X, 'toarray'):
    X_dense = X.toarray()
else:
    X_dense = X

# Overall statistics
total_counts = np.array(X.sum(axis=1)).flatten()
n_genes_detected = np.array((X > 0).sum(axis=1)).flatten()
sparsity = 1.0 - (np.count_nonzero(X_dense) / X_dense.size)

print(f"🔢 EXPRESSION STATISTICS:")
print(f"   Total UMI range: {total_counts.min():,.0f} - {total_counts.max():,.0f}")
print(f"   Mean UMI per spot: {total_counts.mean():,.0f}")
print(f"   Genes detected per spot: {n_genes_detected.min()} - {n_genes_detected.max()}")
print(f"   Mean genes per spot: {n_genes_detected.mean():.0f}")
print(f"   Overall sparsity: {sparsity:.1%}")

In [ ]:
# Create basic distribution plots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Total counts distribution
axes[0,0].hist(total_counts, bins=50, alpha=0.7, edgecolor='black')
axes[0,0].axvline(total_counts.mean(), color='red', linestyle='--', label=f'Mean: {total_counts.mean():.0f}')
axes[0,0].set_xlabel('Total UMI Counts')
axes[0,0].set_ylabel('Frequency')
axes[0,0].set_title('Distribution of Total UMI Counts per Spot')
axes[0,0].legend()

# Genes detected distribution
axes[0,1].hist(n_genes_detected, bins=50, alpha=0.7, edgecolor='black', color='green')
axes[0,1].axvline(n_genes_detected.mean(), color='red', linestyle='--', label=f'Mean: {n_genes_detected.mean():.0f}')
axes[0,1].set_xlabel('Genes Detected')
axes[0,1].set_ylabel('Frequency')
axes[0,1].set_title('Distribution of Genes Detected per Spot')
axes[0,1].legend()

# Scatter plot: UMI vs Genes
axes[1,0].scatter(total_counts, n_genes_detected, alpha=0.6, s=1)
axes[1,0].set_xlabel('Total UMI Counts')
axes[1,0].set_ylabel('Genes Detected')
axes[1,0].set_title('UMI Counts vs Genes Detected')

# Gene expression frequency
gene_detection_freq = np.array((X > 0).sum(axis=0)).flatten()
axes[1,1].hist(gene_detection_freq, bins=50, alpha=0.7, edgecolor='black', color='orange')
axes[1,1].set_xlabel('Number of Spots Expressing Gene')
axes[1,1].set_ylabel('Number of Genes')
axes[1,1].set_title('Gene Detection Frequency')

plt.tight_layout()
plt.show()

## 🗺️ Spatial Coordinates Exploration

Let's examine the spatial organization of our data.

In [ ]:
# Check spatial coordinates
if 'spatial' in adata.obsm:
    coords = adata.obsm['spatial']
    
    print(f"🗺️ SPATIAL COORDINATES:")
    print(f"   Shape: {coords.shape}")
    print(f"   X range: {coords[:, 0].min():.0f} - {coords[:, 0].max():.0f}")
    print(f"   Y range: {coords[:, 1].min():.0f} - {coords[:, 1].max():.0f}")
    
    # Plot spatial layout
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # Basic spatial coordinates
    axes[0].scatter(coords[:, 0], coords[:, 1], s=1, alpha=0.6)
    axes[0].set_xlabel('X Coordinate')
    axes[0].set_ylabel('Y Coordinate')
    axes[0].set_title('Spatial Layout of Spots')
    axes[0].set_aspect('equal')
    
    # Color by total UMI counts
    scatter = axes[1].scatter(coords[:, 0], coords[:, 1], 
                             c=total_counts, s=2, alpha=0.8, cmap='viridis')
    axes[1].set_xlabel('X Coordinate')
    axes[1].set_ylabel('Y Coordinate')
    axes[1].set_title('Spots Colored by Total UMI')
    axes[1].set_aspect('equal')
    plt.colorbar(scatter, ax=axes[1], label='Total UMI')
    
    # Color by genes detected
    scatter2 = axes[2].scatter(coords[:, 0], coords[:, 1], 
                              c=n_genes_detected, s=2, alpha=0.8, cmap='plasma')
    axes[2].set_xlabel('X Coordinate')
    axes[2].set_ylabel('Y Coordinate')
    axes[2].set_title('Spots Colored by Genes Detected')
    axes[2].set_aspect('equal')
    plt.colorbar(scatter2, ax=axes[2], label='Genes Detected')
    
    plt.tight_layout()
    plt.show()
else:
    print("❌ No spatial coordinates found!")

## 🧬 Gene Expression Patterns

Let's explore expression patterns of some key genes.

In [ ]:
# Define gene panels for exploration
gene_panels = {
    'Housekeeping': ['ACTB', 'GAPDH', 'B2M'],
    'Epithelial': ['KRT19', 'KRT8', 'EPCAM', 'CDH1'],
    'Stromal': ['COL1A1', 'VIM', 'ACTA2'],
    'Immune': ['CD68', 'CD3E', 'PTPRC'],
    'Mitochondrial': ['MT-CO1', 'MT-ND1', 'MT-ATP6']
}

# Check which genes are available
available_genes = {}
for panel, genes in gene_panels.items():
    available = [g for g in genes if g in adata.var_names]
    if available:
        available_genes[panel] = available
        print(f"📋 {panel}: {', '.join(available)}")
    else:
        print(f"❌ {panel}: No genes found")

In [ ]:
# Analyze expression patterns for available genes
expression_stats = []

for panel, genes in available_genes.items():
    for gene in genes:
        expr = adata[:, gene].X
        if hasattr(expr, 'toarray'):
            expr = expr.toarray().flatten()
        
        stats = {
            'Gene': gene,
            'Panel': panel,
            'Mean': expr.mean(),
            'Max': expr.max(),
            'Non-zero %': (expr > 0).mean() * 100,
            'CV': expr.std() / expr.mean() if expr.mean() > 0 else 0
        }
        expression_stats.append(stats)

# Create DataFrame
expr_df = pd.DataFrame(expression_stats)
print("🧬 GENE EXPRESSION SUMMARY:")
print(expr_df.round(3))

In [ ]:
# Visualize expression patterns for top variable genes
top_variable_genes = expr_df.nlargest(6, 'CV')['Gene'].tolist()

if len(top_variable_genes) >= 4:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.flatten()
    
    for i, gene in enumerate(top_variable_genes[:4]):
        expr = adata[:, gene].X
        if hasattr(expr, 'toarray'):
            expr = expr.toarray().flatten()
        
        # Histogram
        axes[i].hist(expr, bins=50, alpha=0.7, edgecolor='black')
        axes[i].set_title(f'{gene} Expression Distribution\n(CV: {expr.std()/expr.mean():.3f})')
        axes[i].set_xlabel('Expression Level')
        axes[i].set_ylabel('Frequency')
        
        # Add statistics
        stats_text = f"Mean: {expr.mean():.1f}\nMax: {expr.max():.1f}\nNon-zero: {(expr > 0).mean()*100:.1f}%"
        axes[i].text(0.7, 0.7, stats_text, transform=axes[i].transAxes, 
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
else:
    print("Not enough variable genes found for visualization")

## 🔍 Tissue Detection Analysis

Let's examine how spots are distributed on the tissue.

In [ ]:
# Analyze tissue detection
if 'in_tissue' in adata.obs:
    tissue_spots = adata.obs['in_tissue'].sum()
    total_spots = len(adata.obs)
    
    print(f"🎯 TISSUE DETECTION:")
    print(f"   Total spots: {total_spots:,}")
    print(f"   On tissue: {tissue_spots:,} ({tissue_spots/total_spots*100:.1f}%)")
    print(f"   Off tissue: {total_spots-tissue_spots:,} ({(total_spots-tissue_spots)/total_spots*100:.1f}%)")
    
    # Compare expression between tissue and non-tissue spots
    tissue_data = adata[adata.obs['in_tissue']]
    non_tissue_data = adata[~adata.obs['in_tissue']]
    
    if len(non_tissue_data) > 0:
        tissue_counts = np.array(tissue_data.X.sum(axis=1)).flatten()
        non_tissue_counts = np.array(non_tissue_data.X.sum(axis=1)).flatten()
        
        print(f"")
        print(f"📊 EXPRESSION COMPARISON:")
        print(f"   Tissue spots - Mean UMI: {tissue_counts.mean():.0f}")
        print(f"   Non-tissue spots - Mean UMI: {non_tissue_counts.mean():.0f}")
        
        # Plot comparison
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))
        
        # Histogram comparison
        axes[0].hist(tissue_counts, bins=50, alpha=0.7, label='On tissue', color='green')
        axes[0].hist(non_tissue_counts, bins=50, alpha=0.7, label='Off tissue', color='red')
        axes[0].set_xlabel('Total UMI Counts')
        axes[0].set_ylabel('Frequency')
        axes[0].set_title('UMI Distribution: Tissue vs Non-tissue')
        axes[0].legend()
        
        # Spatial plot with tissue annotation
        if 'spatial' in adata.obsm:
            coords = adata.obsm['spatial']
            tissue_mask = adata.obs['in_tissue'].values
            
            axes[1].scatter(coords[~tissue_mask, 0], coords[~tissue_mask, 1], 
                           s=2, alpha=0.6, color='red', label='Off tissue')
            axes[1].scatter(coords[tissue_mask, 0], coords[tissue_mask, 1], 
                           s=2, alpha=0.6, color='green', label='On tissue')
            axes[1].set_xlabel('X Coordinate')
            axes[1].set_ylabel('Y Coordinate')
            axes[1].set_title('Spatial Distribution: Tissue Detection')
            axes[1].legend()
            axes[1].set_aspect('equal')
        
        plt.tight_layout()
        plt.show()
    else:
        print("   All spots are marked as 'on tissue'")
else:
    print("❌ No tissue detection information found")

## 📊 Summary and Next Steps

Let's summarize our findings and plan the next analysis steps.

In [ ]:
# Summary statistics
print("📋 EXPLORATORY ANALYSIS SUMMARY:")
print("=" * 50)
print(f"📊 Dataset: {adata.n_obs:,} spots × {adata.n_vars:,} genes")
print(f"🎯 Tissue coverage: {adata.obs['in_tissue'].sum():,} spots on tissue")
print(f"🧬 Expression range: {total_counts.min():,.0f} - {total_counts.max():,.0f} UMI per spot")
print(f"📈 Sparsity: {sparsity:.1%}")
print(f"🔬 Available gene panels: {len(available_genes)}")

print(f"\n🎯 RECOMMENDED NEXT STEPS:")
print(f"   1. Quality control with filtering")
print(f"   2. Normalization and preprocessing")
print(f"   3. Spatial clustering analysis")
print(f"   4. Marker gene identification")
print(f"   5. Biological interpretation")

if 'artificial_variation_fix' in adata.uns:
    print(f"\n⚠️ NOTE: Artificial uniformity was detected and corrected")
    print(f"   This suggests the original data was synthetic or over-processed")

print(f"\n✅ Exploratory analysis complete!")
print(f"   Continue with notebook 02_quality_control.ipynb")

In [ ]:
# Save processed data for next notebook
output_path = "../results/adata_explored.h5ad"
adata.write(output_path)
print(f"💾 Data saved to: {output_path}")